In [1]:
%%javascript
document.title='Jupyter Lab - Oil prices.ipynb'

<IPython.core.display.Javascript object>

In [156]:
from glob import glob
from tqdm import tqdm  
import pandas as pd
import numpy as np
import os
 

In [111]:
def freq_string_generator(date_dict):
    freq_map = {"seconds" : 'S', 'minutes' : 'T', 'days': 'D', 'months': 'M'}
    freq_string = ''
    
    for k in freq_map:
        if k in date_dict:
            freq_string += str(date_dict[k]) + freq_map[k]
    
    return freq_string

In [57]:
%%time
if os.path.exists('oil_dataset.csv'):
    print('Loading dataset from csv')
    df = pd.read_csv('oil_dataset.csv', index_col=0)
    df.index = pd.to_datetime(df.index)
else:
    DATASET_PATH = './data'
    files = [file for file in glob(DATASET_PATH + '/*') if '.' not in file.split('/')[-1]]
    df = []
    for file in tqdm(files):
        df_ = pd.read_csv(file, header=None)
        df_.columns = [0,1,'price','volume']
        df_['date'] = pd.to_datetime(df_[0] + ' ' + df_[1])
        df_ = df_[['date','price','volume']].set_index('date')
        df.append(df_)
    df = pd.concat(df).sort_index()
    df.to_csv('oil_dataset.csv')

Loading dataset from csv
CPU times: user 2min 56s, sys: 2min 54s, total: 5min 51s
Wall time: 7min 44s


In [121]:
df_test = df.iloc[:df.shape[0] // 50]
df_test.index = pd.to_datetime(df_test.index).to_period('S')
df_test = df_test.loc[:,['price']]
df_test

,price
date,
2010-01-03 18:00:00,79.63
2010-01-03 18:00:00,79.63
2010-01-03 18:00:00,79.63
2010-01-03 18:00:00,79.63
2010-01-03 18:00:00,79.63
...,...
2010-04-01 12:20:56,84.90
2010-04-01 12:20:56,84.89
2010-04-01 12:20:56,84.89


In [161]:
# df_test.resample('T').first()
tt = df_test.resample('60T').agg(['first', 'last'])
tt.columns = tt.columns.droplevel(0)
tt['delta'] = tt['last']/tt['first'] - 1
tt

,first,last,delta
date,,,
2010-01-03 18:00,79.63,79.94,0.003893
2010-01-03 19:00,79.95,79.96,0.000125
2010-01-03 20:00,79.99,79.91,-0.001000
2010-01-03 21:00,79.90,79.95,0.000626
2010-01-03 22:00,79.96,79.99,0.000375
...,...,...,...
2010-04-01 08:00,84.58,84.81,0.002719
2010-04-01 09:00,84.77,84.94,0.002005
2010-04-01 10:00,85.00,84.72,-0.003294


In [163]:
tt.shift(-1)

,first,last,delta
date,,,
2010-01-03 18:00,79.95,79.96,0.000125
2010-01-03 19:00,79.99,79.91,-0.001000
2010-01-03 20:00,79.90,79.95,0.000626
2010-01-03 21:00,79.96,79.99,0.000375
2010-01-03 22:00,79.97,80.00,0.000375
...,...,...,...
2010-04-01 08:00,84.77,84.94,0.002005
2010-04-01 09:00,85.00,84.72,-0.003294
2010-04-01 10:00,84.72,84.97,0.002951


In [137]:
tt

,first,last
date,,
2010-01-03 18:00,79.63,79.82
2010-01-03 18:01,79.86,79.95
2010-01-03 18:02,79.96,79.93
2010-01-03 18:03,79.94,79.90
2010-01-03 18:04,79.89,79.89
...,...,...
2010-04-01 12:16,84.80,84.85
2010-04-01 12:17,84.85,84.84
2010-04-01 12:18,84.84,84.82


In [160]:
tt

,first,last,delta
date,,,
2010-01-06 11:21,81.95,82.67,0.008786
2010-02-03 14:26,76.61,77.08,0.006135
2010-02-05 08:30,72.62,73.11,0.006747
2010-02-05 11:30,71.84,72.20,0.005011
2010-02-10 13:17,74.23,74.62,0.005254
2010-02-18 09:29,77.89,78.45,0.007190
2010-03-29 09:38,81.43,81.88,0.005526


In [166]:
def resample(df, freq_dict, delta_threshold = 0.005):
    
    if isinstance(freq_dict, str):
        freq_string = freq_dict
    else:
        freq_map = {"seconds" : 'S', 'minutes' : 'T', 'days': 'D', 'months': 'M'}
        freq_string = ''

        for k in freq_map:
            if k in freq_dict and freq_dict[k]:
                freq_string += str(freq_dict[k]) + freq_map[k]
            
    resampled = df.resample(freq_string).agg(['first', 'last'])
    resampled.columns = resampled.columns.droplevel(0)
    resampled['delta'] = resampled['last']/resampled['first'] - 1
    resampled['delta_next_day'] = resampled['delta'].shift(-1)
    resampled = resampled.iloc[:-1]
    resampled = resampled.loc[np.abs(resampled['delta']) > 0.95*delta_threshold]
            
    return resampled
    

In [167]:
resample(df_test,dict(seconds=0, minutes=1), delta_threshold=0.005)

,first,last,delta,delta_next_day
date,,,,
2010-01-06 10:30,82.16,81.49,-0.008155,-0.000736
2010-01-06 11:21,81.95,82.67,0.008786,-0.003145
2010-01-12 16:30,80.55,80.16,-0.004842,-0.001123
2010-01-13 10:30,79.06,78.47,-0.007463,0.002039
2010-01-21 11:09,77.12,76.49,-0.008169,0.000000
2010-01-27 10:32,74.57,74.13,-0.005900,-0.000270
2010-01-27 13:17,73.95,73.51,-0.005950,-0.001904
2010-01-29 11:59,73.70,73.30,-0.005427,0.002321
2010-02-03 14:26,76.61,77.08,0.006135,-0.000649


,first,last,delta
date,,,
2010-01-06 11:21,81.95,82.67,0.008786
2010-02-03 14:26,76.61,77.08,0.006135
2010-02-05 08:30,72.62,73.11,0.006747
2010-02-05 11:30,71.84,72.20,0.005011
2010-02-10 13:17,74.23,74.62,0.005254
2010-02-18 09:29,77.89,78.45,0.007190
2010-03-29 09:38,81.43,81.88,0.005526


In [107]:
df_test.resample('T').agg({"price": "mean", "volume": "sum"})

,price,volume
date,,
2010-01-03 18:00,79.801935,112
2010-01-03 18:01,79.899429,126
2010-01-03 18:02,79.914906,109
2010-01-03 18:03,79.908857,49
2010-01-03 18:04,79.881538,15
...,...,...
2010-04-01 12:16,84.815792,392
2010-04-01 12:17,84.850631,199
2010-04-01 12:18,84.835060,249


PeriodIndex(['2010-01-03 18:00:00', '2010-01-03 18:00:00',
             '2010-01-03 18:00:00', '2010-01-03 18:00:00',
             '2010-01-03 18:00:00', '2010-01-03 18:00:00',
             '2010-01-03 18:00:00', '2010-01-03 18:00:00',
             '2010-01-03 18:00:00', '2010-01-03 18:00:00',
             ...
             '2010-04-01 12:20:56', '2010-04-01 12:20:56',
             '2010-04-01 12:20:56', '2010-04-01 12:20:56',
             '2010-04-01 12:20:56', '2010-04-01 12:20:56',
             '2010-04-01 12:20:56', '2010-04-01 12:20:56',
             '2010-04-01 12:20:56', '2010-04-01 12:20:56'],
            dtype='period[S]', name='date', length=8219999, freq='S')

In [75]:
pd.to_datetime(df_test.index)

DatetimeIndex(['2010-01-03 18:00:00', '2010-01-03 18:00:00',
               '2010-01-03 18:00:00', '2010-01-03 18:00:00',
               '2010-01-03 18:00:00', '2010-01-03 18:00:00',
               '2010-01-03 18:00:00', '2010-01-03 18:00:00',
               '2010-01-03 18:00:00', '2010-01-03 18:00:00',
               ...
               '2010-04-01 12:20:56', '2010-04-01 12:20:56',
               '2010-04-01 12:20:56', '2010-04-01 12:20:56',
               '2010-04-01 12:20:56', '2010-04-01 12:20:56',
               '2010-04-01 12:20:56', '2010-04-01 12:20:56',
               '2010-04-01 12:20:56', '2010-04-01 12:20:56'],
              dtype='datetime64[ns]', name='date', length=8219999, freq=None)

In [ ]:
df.index.dtype